In [1]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_excel('no_missing.xlsx', converters={'Merchnum': lambda x: str(x), 'Merch_zip': lambda x : str(x)})

In [ ]:
df = df.drop(['Unnamed: 0','assign','minus'],axis = 1)

In [ ]:
df.info()

## 1. Benford's Law Variable for merchnum and cardnum

In [ ]:
old = df.copy()

In [ ]:
old['stramount'] = old['Amount'].astype('str')

In [ ]:
old['stramount'] = old['stramount'].str.replace('.','')
old['stramount'] = old['stramount'].astype(int).astype('str')

In [ ]:
old[old['stramount'].str.startswith('0')]

In [ ]:
old['Merch_description'] = old['Merch_description'].str.lower()

In [ ]:
old = old[~old['Merch_description'].str.contains('fedex')]

In [ ]:
old['digit_1'] = old['stramount'].str[0]

In [ ]:
old['digit_1'] = old['digit_1'].astype(int)

In [ ]:
old = old[['Recnum','Merchnum','Cardnum','digit_1']]

In [ ]:
old['n_low'] = old['digit_1'] <= 2
old['n_high'] = old['digit_1'] > 2

In [ ]:
new_card = old[['Recnum','Cardnum']]
new_merch = old[['Recnum','Merchnum']]

### 1.1 For each Cardnum

In [ ]:
benford_card = pd.merge(new_card, old, left_on = 'Cardnum', right_on = 'Cardnum')

In [ ]:
benford_card = benford_card[benford_card['Recnum_x'] >= benford_card['Recnum_y']]

In [ ]:
benford_card.head()

In [ ]:
benford_card = benford_card.groupby(['Recnum_x','Cardnum']).agg({'n_low':'sum','n_high':'sum'})

In [ ]:
benford_card.loc[(235,5142183973),]

In [ ]:
benford_card.head()

In [ ]:
c = 3
n_mid = 15

In [ ]:
benford_card['n_low'] = benford_card['n_low'].replace(0,1)
benford_card['n_high'] = benford_card['n_high'].replace(0,1)
benford_card['n'] = benford_card['n_low'] + benford_card['n_high']
benford_card['R'] = benford_card['n_low']/benford_card['n_high'] *1.096
benford_card['1/R'] = 1/benford_card['R']
benford_card['t'] = (benford_card['n'] - n_mid)/c
benford_card['U']  = benford_card[['R','1/R']].max(axis = 1)
benford_card['U_star_card'] = (benford_card['U'] -1)/(1+np.exp(-benford_card['t'])) +1

In [ ]:
benford_card = benford_card.reset_index()

In [ ]:
benford_card

### 1.2 For each Merchnum

In [ ]:
benford_merch = pd.merge(new_merch, old, left_on = 'Merchnum', right_on = 'Merchnum')
benford_merch = benford_merch[benford_merch['Recnum_x'] >= benford_merch['Recnum_y']]
benford_merch = benford_merch.groupby(['Recnum_x','Merchnum']).agg({'n_low':'sum','n_high':'sum'})
benford_merch['n_low'] = benford_merch['n_low'].replace(0,1)
benford_merch['n_high'] = benford_merch['n_high'].replace(0,1)
benford_merch['n'] = benford_merch['n_low'] + benford_merch['n_high']
benford_merch['R'] = benford_merch['n_low']/benford_merch['n_high'] *1.096
benford_merch['1/R'] = 1/benford_merch['R']
benford_merch['t'] = (benford_merch['n'] - n_mid)/c
benford_merch['U']  = benford_merch[['R','1/R']].max(axis = 1)
benford_merch['U_star_merch'] = (benford_merch['U'] -1)/(1+np.exp(-benford_merch['t'])) +1
benford_merch = benford_merch.reset_index()

In [ ]:
benford_merch

In [ ]:
benford_value = pd.merge(benford_card[['Recnum_x','U_star_card']], benford_merch[['Recnum_x','U_star_merch']], left_on = 'Recnum_x', right_on = 'Recnum_x')

In [ ]:
benford_value.columns = ['Recnum','U_star_card','U_star_merch']

In [ ]:
benford_value = benford_value.set_index('Recnum')

In [ ]:
benford = df[['Recnum']].set_index('Recnum')

In [ ]:
benford = pd.merge(benford, benford_value, left_index = True, right_index = True, how = 'left')

In [ ]:
benford['U_star_card'] = benford['U_star_card'].fillna(1)
benford['U_star_merch'] = benford['U_star_merch'].fillna(1)

In [ ]:
benford